In [1]:
import tdmclient.notebook
await tdmclient.notebook.start()
from tdmclient import ClientAsync, aw
client = ClientAsync()
client.process_waiting_messages()
node = await client.wait_for_node()
aw(node.run())
aw(node.stop())
aw(node.unlock())

import time
from l42 import *
from local_nav import *
from VisGraph import *
from update_pos import *


In [2]:
#getters and setters functions

@tdmclient.notebook.sync_var
def get_prox_value():
    global prox_horizontal
    return prox_horizontal

#@tdmclient.notebook.sync_var
#def get_ground_value():
#    global prox_ground_reflected
#    return prox_ground_reflected

@tdmclient.notebook.sync_var
def get_left_speed():
    global motor_left_target
    return motor_left_target

@tdmclient.notebook.sync_var
def get_right_speed():
    global motor_right_target
    return motor_right_target

@tdmclient.notebook.sync_var
def set_motor_speed(left_speed, right_speed):
    global motor_left_target, motor_right_target
    motor_left_target = left_speed
    motor_right_target = right_speed

In [ ]:
#Create constant variables
next_goal_trigger = 20

#Creation of the map and the global path
global_path = initialisation()
start = global_path[0]
stop = global_path[len(global_path)-1]

#Set the initial motor speeds at 0
set_motor_speed(0, 0)

#Initialisation of the variables
prox_sensors = np.array([0, 0, 0, 0, 0, 0, 0])

#Find the initial position and orientation of the robot
position, orientation = detect_thymio()

#Initialisation of the variables that will keep in memory the position and orientation of the last iteration
old_position = position
old_orientation = orientation

#Initialisation of the variables that will keep track of the current goal position and orientation
current_goal = 1
current_goal_pos = global_path[current_goal]
print(position)

distance_to_goal, current_goal_orientation = calculation_distance_and_angle(position, current_goal_pos)

#Initialisation of the variables that will be updated to set the speed of the wheels
left_speed = 0
right_speed = 0

#Initialisation of the boolean that will recall if the camera detection is possible or not at each iteration
possible_detection = True

while(True):
    print(current_goal)
    current_goal_pos = global_path[current_goal]
    #Recall the values of the sensors and the wheel speeds 
    prox_sensors = get_prox_value()
    #prox_ground_values = get_ground_value()
    #set_motor_speed(100, 100)
    
    #Reset and update the detection boolean
    #possible_detection = #fonction prévue à cet effet
    
    #if possible_detection :
        #Vision : Returns the position and the orientation of the robot from the camera
    position, orientation = detect_thymio()
    print(position)
    
    #else :
        #The position and orientation of the robot is estimated from its last state and the wheel speeds
        #position = kalman(position, orientation, left_speed, right_speed) #Ou peu importe la forme de la fonction 
        #orientation = kalman(old_position, orientation, left_speed, right_speed) #Ou peu importe la forme de la fonction
    
    #Update the current goal orientation and the distance from the robot
    distance_to_goal, current_goal_orientation = calculation_distance_and_angle(position, current_goal_pos)

    #Find the speeds to apply to the robot
    left_speed, right_speed = localNavigation(orientation, old_orientation, current_goal_orientation, prox_sensors)
    #Apply the desired speeds to the robot
    set_motor_speed(int(left_speed), int(right_speed))
    
    #Keep in memory the position and orientation of the robot for the next 
    old_position = position
    old_orientation = orientation
    
    #Update the current goal if necessary
    if (distance_to_goal < next_goal_trigger) :
        current_goal += 1
    
    if (current_goal == len(global_path)) :
        set_motor_speed(0, 0)
        break
    
    await client.sleep(0.1)
  




No detection...
No detection...
No detection...
No detection...
No detection...
No detection...
No detection...
No detection...
No detection...
No detection...
No detection...
No detection...
No detection...
No detection...
No detection...
No detection...
No detection...
No detection...
No detection...
No detection...
No detection...
No detection...
No detection...
No detection...
No detection...
No detection...
No detection...
No detection...


100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 117.69it/s]


No thymio
[62 31]
1
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
[66 24]
[0, 0, 0, 0, 0, 0, 4514]
1
No thymio
No thymio
No thymio
[61 33]
[0, 0, 0, 0, 0, 0, 4514]
1
No thymio
[69 24]
[0, 0, 0, 0, 0, 0, 4514]
1
No thymio
No thymio
[67 39]
[0, 0, 0, 0, 0, 0, 4514]
1
No thymio
[70 27]
[0, 0, 0, 0, 0, 0, 4520]
1
No thymio
No thymio
[71 35]
[0, 0, 0, 0, 0, 0, 4520]
1
No thymio
[73 39]
[0, 0, 0, 0, 0, 0, 4513]
1
No thymio
No thymio
[77 46]
[0, 0, 0, 0, 0, 0, 4513]
1
No thymio
[80 42]
[0, 0, 0, 0, 0, 0, 4513]
1
No thymio
No thymio
[84 44]
[0, 0, 0, 0, 0, 0, 4508]
1
No thymio
[87 45]
[0, 0, 0, 0, 0, 0, 4508]
1
No thymio
[90 47]
[0, 0, 0, 0, 0, 0, 4508]
1
No thymio
[98 54]
[0, 0, 0, 0, 0, 0, 4508]
1
No thymio
[94 55]
[0, 0, 0, 1903, 0, 0, 4508]
1
No thymio
[99 51]
[0, 0, 0, 19, 0, 0, 4508]
1
No thymio
[102  51]
[0, 0, 0, 1903, 0, 0, 4508]
1
No thymio
[105  53]
[0, 0, 0, 19, 0, 0, 4508]
1
No thymio
[108  54]
[0, 0, 0, 4259, 0, 0, 4508]
1
No thymio
[11

C:\Users\maxim\Desktop\EPFL\MA1\Basics of mobile robotics\project_me\local_nav.py:35: RuntimeWarning: divide by zero encountered in double_scalars
  angle = np.around(np.arctan((position_goal[1] - position_thymio[1]) / (position_goal[0] - position_thymio[0])), 3)


2
No thymio
[309 156]
[0, 0, 0, 0, 0, 0, 4511]
2
No thymio
[310 160]
[0, 0, 0, 0, 0, 0, 4511]
2
No thymio
[311 164]
[0, 0, 0, 0, 0, 0, 4511]
2
No thymio
[311 169]
[0, 0, 0, 0, 0, 0, 4511]
2
No thymio
[312 173]
[0, 0, 0, 0, 0, 0, 4511]
2
No thymio
[312 177]
[0, 0, 0, 0, 0, 0, 4508]
2
No thymio
[312 181]
[0, 0, 0, 0, 0, 0, 4508]
2
No thymio
[313 184]
[0, 0, 0, 0, 0, 0, 4508]
2
No thymio
[313 188]
[0, 0, 0, 0, 0, 0, 4509]
2
No thymio
[313 194]
[0, 0, 0, 0, 0, 0, 4511]
3
No thymio
[313 198]
[0, 0, 0, 0, 0, 0, 4511]
3
No thymio
[313 202]
[0, 0, 0, 0, 0, 0, 4511]
3
No thymio
[313 207]
[0, 0, 0, 0, 0, 0, 4511]
3
No thymio
[313 211]
[0, 0, 0, 0, 0, 0, 4516]
3
No thymio
[312 216]
[0, 0, 0, 0, 0, 0, 4514]
3
No thymio
[313 220]
[0, 0, 0, 0, 0, 0, 4514]
3
No thymio
[314 223]
[0, 0, 0, 0, 0, 0, 4514]
3
No thymio
[315 226]
[0, 0, 0, 0, 0, 0, 4514]
3
No thymio
[317 229]
[0, 0, 0, 0, 0, 0, 4508]
3
No thymio
[320 233]
[0, 0, 0, 0, 0, 0, 4508]
3
No thymio
[322 236]
[0, 0, 0, 0, 0, 0, 4512]
3
No thymio
[

4
No thymio
[533 350]
[0, 0, 0, 0, 0, 0, 4512]
4
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
[537 355]
[0, 0, 0, 0, 0, 0, 4512]
4
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No t

No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio


No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio


No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio


In [ ]:
await tdmclient.notebook.stop()